## Preambule

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import xarray as xr

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_10200\2088128098.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Input

In [2]:
params = dict(start_year_analysis=2021)
path_alloc = "K:/ECEMF/T5.2/Allocations/"
path_save = "K:/ECEMF/T5.2/"

## Read

In [3]:
xr_dataread = xr.open_dataset(path_save + "xr_dataread.nc", engine='netcdf4')

In [4]:
focusregs = np.array(xr_dataread.Region)
temperatures = [1.5, 2.0]
rules = ['GF', 'PC', 'PCC', 'ECPC', 'AP', 'GDR']

## Save allocations by year

In [5]:
for year in np.array([2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100]):
    print("Starting with", year)
    for cty_i, cty in tqdm(enumerate(focusregs)):
        ds = xr.open_dataset(path_alloc + "xr_alloc_"+cty+".nc").sel(Time=year).expand_dims(Region=[cty])
        if cty_i == 0:
            xrt = ds.copy()
        else:
            xrt = xr.merge([xrt, ds])
        ds.close()
    xrt.astype("float32").to_netcdf(path_save + "xr_alloc_"+str(year)+".nc", format='NETCDF4')

Starting with 2030


0it [00:00, ?it/s]

207it [15:20,  4.45s/it]


Starting with 2040


207it [11:47,  3.42s/it]


...

Starting with 2090


207it [11:27,  3.32s/it]


Starting with 2100


73it [01:45,  1.45s/it]


KeyboardInterrupt: 

Trajectory emissions

In [ ]:
for temp_i, temp in enumerate([1.5, 2.0]):
    for cty_i, cty in tqdm(enumerate(focusregs)):
        ds = xr.open_dataset(path_alloc + "xr_alloc_"+cty+".nc").sel(Temperature=temp, Risk=[0.5, 0.67][temp_i], Time=np.arange(params['start_year_analysis'], 2101)).expand_dims(Region=[cty])
        if cty_i == 0:
            xrt = ds.copy()
        else:
            xrt = xr.merge([xrt, ds])
        ds.close()
    xrt.astype("float32").to_netcdf(path_save + "xr_traj_"+str(temp)[0]+str(temp)[2]+"_50.nc", format='NETCDF4')

207it [20:38,  5.99s/it]
207it [20:19,  5.89s/it]
